In [0]:
%pip install yfinance

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 143.6 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15548 sha256=19407041bef0383bdd1028a7b3bd5cf8dfb0a62315bf1640d967c78f4aefc022
  Stored in directory: /home/spark-b82b0bbf-3dc2-4d5f-aeac-6d/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
  Created wheel for peewee: filename=peewee-3.18.3-cp312-cp312-linux_aarch64.whl size=909445 sha256=6db08115

In [0]:
%pip install openpyxl

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
"""
Stock Portfolio Analysis Script
Fetches current stock prices and calculates portfolio performance
Outputs results to an Excel file
"""

import pandas as pd
import yfinance as yf
from datetime import datetime


In [0]:
def get_stock_data(ticker):
    """Fetch current stock price and basic info"""
    try:
        stock = yf.Ticker(ticker)
        current_price = stock.info.get("currentPrice") or stock.info.get(
            "regularMarketPrice"
        )
        if current_price is None:
            # Try getting from history if info doesn't have price
            hist = stock.history(period="1d")
            if not hist.empty:
                current_price = hist["Close"].iloc[-1]
        return current_price
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None


In [0]:
def export_to_excel(results, filename=None):
    """Export portfolio analysis to Excel file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"portfolio_analysis_{timestamp}.xlsx"

    df = pd.DataFrame(results)

    # Sort by Profit/Loss (descending - best performers first)
    # Handle 'N/A' values by treating them as 0 for sorting
    df_sorted = df.copy()
    df_sorted["PL_Sort"] = df_sorted["Profit/Loss"].apply(
        lambda x: x if x != "N/A" else -999999
    )
    df_sorted = df_sorted.sort_values("PL_Sort", ascending=False)
    df_sorted = df_sorted.drop("PL_Sort", axis=1)
    df = df_sorted

    # Calculate totals
    total_initial = df[df["Initial Value"] != "N/A"]["Initial Value"].sum()
    total_current = df[df["Current Value"] != "N/A"]["Current Value"].sum()
    total_pl = df[df["Profit/Loss"] != "N/A"]["Profit/Loss"].sum()
    total_return_pct = (
        ((total_current - total_initial) / total_initial) * 100
        if total_initial > 0
        else 0
    )

    # Add summary row
    summary = pd.DataFrame(
        [
            {
                "Ticker": "TOTAL",
                "Shares": "",
                "Purchase Price": "",
                "Current Price": "",
                "Initial Value": round(total_initial, 2),
                "Current Value": round(total_current, 2),
                "Profit/Loss": round(total_pl, 2),
                "Return %": round(total_return_pct, 2),
            }
        ]
    )

    df = pd.concat([df, summary], ignore_index=True)

    # Create Excel writer with formatting
    with pd.ExcelWriter(filename, engine="openpyxl") as writer:
        df.to_excel(writer, sheet_name="Portfolio", index=False)

        # Get the worksheet
        worksheet = writer.sheets["Portfolio"]

        # Adjust column widths
        for column in worksheet.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = min(max_length + 2, 20)
            worksheet.column_dimensions[column_letter].width = adjusted_width

        # Apply formatting to header
        from openpyxl.styles import Font, PatternFill, Alignment

        header_fill = PatternFill(
            start_color="366092", end_color="366092", fill_type="solid"
        )
        header_font = Font(bold=True, color="FFFFFF")

        for cell in worksheet[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.alignment = Alignment(horizontal="center")

        # Format summary row
        last_row = worksheet.max_row
        summary_fill = PatternFill(
            start_color="D9E1F2", end_color="D9E1F2", fill_type="solid"
        )
        summary_font = Font(bold=True)

        for cell in worksheet[last_row]:
            cell.fill = summary_fill
            cell.font = summary_font

    print(f"\n✓ Portfolio analysis exported to: {filename}")
    return filename, df

In [0]:
def analyze_portfolio(portfolio_data):
    """
    Analyze portfolio performance

    Parameters:
    portfolio_data: list of dicts with keys: 'ticker', 'shares', 'purchase_price'
    """
    results = []

    for holding in portfolio_data:
        ticker = holding["ticker"]
        shares = holding["shares"]
        purchase_price = holding["purchase_price"]

        print(f"Fetching data for {ticker}...")
        current_price = get_stock_data(ticker)

        if current_price:
            initial_value = shares * purchase_price
            current_value = shares * current_price
            profit_loss = current_value - initial_value
            profit_loss_pct = ((current_price - purchase_price) / purchase_price) * 100

            results.append(
                {
                    "Ticker": ticker,
                    "Shares": shares,
                    "Purchase Price": purchase_price,
                    "Current Price": round(current_price, 2),
                    "Initial Value": round(initial_value, 2),
                    "Current Value": round(current_value, 2),
                    "Profit/Loss": round(profit_loss, 2),
                    "Return %": round(profit_loss_pct, 2),
                }
            )
        else:
            results.append(
                {
                    "Ticker": ticker,
                    "Shares": shares,
                    "Purchase Price": purchase_price,
                    "Current Price": "N/A",
                    "Initial Value": round(shares * purchase_price, 2),
                    "Current Value": "N/A",
                    "Profit/Loss": "N/A",
                    "Return %": "N/A",
                }
            )

    return results

In [0]:
# EXAMPLE PORTFOLIO - Replace with your actual holdings
portfolio = [
{"ticker": "ITGR", "shares": 20, "purchase_price": 68.85},
{"ticker": "ZIM", "shares": 15, "purchase_price": 17.63},
{"ticker": "SMCI", "shares": 11, "purchase_price": 46.28},
{"ticker": "LULU", "shares": 9, "purchase_price": 175.83},
{"ticker": "V", "shares": 4, "purchase_price": 343.19},
{"ticker": "UNH", "shares": 4, "purchase_price": 319.76},
{"ticker": "REGN", "shares": 3, "purchase_price": 570.33},
{"ticker": "META", "shares": 2, "purchase_price": 664.25},
{"ticker": "MA", "shares": 2, "purchase_price": 558.50},

# {'ticker': 'VUSA', 'shares': 4, 'purchase_price': 84.69},
# {'ticker': 'AAPL', 'shares': 10, 'purchase_price': 150.00},
# {'ticker': 'MSFT', 'shares': 5, 'purchase_price': 300.00},
# {'ticker': 'GOOGL', 'shares': 8, 'purchase_price': 120.00},
# {'ticker': 'TSLA', 'shares': 3, 'purchase_price': 250.00},
# {'ticker': 'AMZN', 'shares': 4, 'purchase_price': 140.00},
]

print("=" * 60)
print("STOCK PORTFOLIO ANALYSIS")
print("=" * 60)
print(f"\nAnalyzing {len(portfolio)} holdings...\n")

# Analyze portfolio
results = analyze_portfolio(portfolio) # list

STOCK PORTFOLIO ANALYSIS

Analyzing 9 holdings...

Fetching data for ITGR...
Fetching data for ZIM...
Fetching data for SMCI...
Fetching data for LULU...
Fetching data for V...
Fetching data for UNH...
Fetching data for REGN...
Fetching data for META...
Fetching data for MA...


In [0]:
df_results = pd.DataFrame(results)
df_results["PL_Sort"] = df_results["Profit/Loss"].apply(lambda x: x if x != "N/A" else -999999)
df_results = df_results.sort_values("PL_Sort", ascending=False).drop("PL_Sort", axis=1)
display(df_results)

Ticker,Shares,Purchase Price,Current Price,Initial Value,Current Value,Profit/Loss,Return %
REGN,3,570.33,674.55,1710.99,2023.65,312.66,18.27
UNH,4,319.76,327.45,1279.04,1309.8,30.76,2.4
ITGR,20,68.85,69.82,1377.0,1396.4,19.4,1.41
MA,2,558.5,558.35,1117.0,1116.7,-0.3,-0.03
V,4,343.19,338.92,1372.76,1355.68,-17.08,-1.24
ZIM,15,17.63,15.41,264.45,231.15,-33.3,-12.59
LULU,9,175.83,169.81,1582.47,1528.29,-54.18,-3.42
META,2,664.25,627.08,1328.5,1254.16,-74.34,-5.6
SMCI,11,46.28,38.82,509.08,427.02,-82.06,-16.12


In [0]:
# Calculate totals for columns from 'Purchase Price' to 'Return %'
totals_row = {
    "Ticker": "TOTALS",
    "Shares": round(df_results[df_results["Shares"] != "N/A"]["Shares"].sum(), 2),
    "Purchase Price": "N/A",
    "Current Price": "N/A",
    "Initial Value": round(df_results[df_results["Initial Value"] != "N/A"]["Initial Value"].sum(), 2),
    "Current Value": round(df_results[df_results["Current Value"] != "N/A"]["Current Value"].sum(), 2),
    "Profit/Loss": round(df_results[df_results["Profit/Loss"] != "N/A"]["Profit/Loss"].sum(), 2),
    "Return %": round(df_results[df_results["Return %"] != "N/A"]["Return %"].sum(), 2),
}

df_totals = pd.DataFrame([totals_row])
# df_results_with_totals = pd.concat([df_results, df_totals], ignore_index=True)
display(df_totals)

Ticker,Shares,Purchase Price,Current Price,Initial Value,Current Value,Profit/Loss,Return %
TOTALS,70,N/A,N/A,10541.29,10642.85,101.56,-16.92


In [0]:
export_to_excel(results)


✓ Portfolio analysis exported to: portfolio_analysis_20251112_132627.xlsx


('portfolio_analysis_20251112_132627.xlsx',
   Ticker Shares Purchase Price  ... Current Value  Profit/Loss  Return %
 0   REGN      3         570.33  ...       2023.65       312.66     18.27
 1    UNH      4         319.76  ...       1309.80        30.76      2.40
 2   ITGR     20          68.85  ...       1396.40        19.40      1.41
 3     MA      2          558.5  ...       1116.70        -0.30     -0.03
 4      V      4         343.19  ...       1355.68       -17.08     -1.24
 5    ZIM     15          17.63  ...        231.15       -33.30    -12.59
 6   LULU      9         175.83  ...       1528.29       -54.18     -3.42
 7   META      2         664.25  ...       1254.16       -74.34     -5.60
 8   SMCI     11          46.28  ...        427.02       -82.06    -16.12
 9  TOTAL                        ...      10642.85       101.56      0.96
 
 [10 rows x 8 columns])